Q-3. Imagine you have a dataset where you have different categories of data, Now
you need to find the most similar data to the given data by using any 4 different
similarity algorithms. Now you have to build a model which can find the most similar
data to the given data.
Dataset This is the Dataset You can use this dataset for this question.

In [4]:
import json

data = []
with open("News_Category_Dataset_v3.json", "r") as file:
    for line in file:
        data.append(json.loads(line))

df = pd.DataFrame(data)


In [5]:
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   link               209527 non-null  object
 1   headline           209527 non-null  object
 2   category           209527 non-null  object
 3   short_description  209527 non-null  object
 4   authors            209527 non-null  object
 5   date               209527 non-null  object
dtypes: object(6)
memory usage: 9.6+ MB


In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances_chunked
from scipy.spatial.distance import jaccard

# Select relevant columns
selected_columns = ['headline', 'category', 'short_description', 'authors', 'date']
data_selected = df.loc[:, selected_columns]

# Combine text columns into a single column
data_selected['combined_text'] = data_selected[selected_columns[1:]].apply(lambda x: ' '.join(x.dropna()), axis=1)

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data_selected['combined_text'])

# Define a function to compute Jaccard similarity
def jaccard_similarity_sparse(X, Y):
    return pairwise_distances_chunked(X, Y, metric=jaccard)

# Define a function to find similar data based on the similarity algorithm
def find_similar_data(input_data, similarity_algorithm):
    if similarity_algorithm == 'euclidean':
        similarity_scores = pairwise_distances_chunked(input_data, tfidf_matrix, metric='euclidean')
    elif similarity_algorithm == 'manhattan':
        similarity_scores = pairwise_distances_chunked(input_data, tfidf_matrix, metric='manhattan')
    elif similarity_algorithm == 'jaccard':
        similarity_scores = jaccard_similarity_sparse(input_data, tfidf_matrix)
    else:
        return None

    # Accumulate the similarity scores in a list
    scores_list = []
    for scores in similarity_scores:
        scores_list.append(scores)

    # Concatenate the similarity scores
    similarity_scores = pd.DataFrame(sum(scores_list, []))

    # Find the indices of the most similar data
    most_similar_indices = similarity_scores.values.argsort(axis=0)[:10]

    # Get the most similar data based on the indices
    most_similar_data = data_selected.iloc[most_similar_indices.flatten()]

    return most_similar_data

# Provide the input data for similarity search
input_data = df['short_description']  # Replace with your input data

# Find the most similar data using cosine similarity
cosine_similar_data = find_similar_data(tfidf_vectorizer.transform(input_data), 'cosine')

# Find the most similar data using Euclidean distance
euclidean_similar_data = find_similar_data(tfidf_vectorizer.transform(input_data), 'euclidean')

# Find the most similar data using Manhattan distance
manhattan_similar_data = find_similar_data(tfidf_vectorizer.transform(input_data), 'manhattan')

# Find the most similar data using Jaccard similarity
jaccard_similar_data = find_similar_data(tfidf_vectorizer.transform(input_data), 'jaccard')
